# Imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
from collections import defaultdict
from time import strftime, gmtime
import random
from random import sample

import string

# Params

In [2]:
subject = 0
video_dir = "robot_stimuli"
cdir = os.getcwd()
subject_dir = f"{cdir}\\data\\subject_{subject}"

# Signs

In [3]:
def config_to_name(configuration):
    return f"{configuration['laterality']}_{configuration['localisation']}_{configuration['movement']}_{configuration['direction']}"

def name_to_config(fname):
    return dict(zip(['laterality', 'localisation', 'movement', 'direction'], fname.split('_')))

In [4]:
configurations = {
    'laterality' : ['L', 'R'],
    'localisation' : ['Head', 'Arm', 'Torso', 'Base'],
    'movement' : ['X', 'Y', 'Z'],
    'direction' : ['For', 'Back']
    }

all_configurations = list(itertools.product(*configurations.values()))

In [5]:
row_list = []

all_configurations = list(itertools.product(*configurations.values()))

for configuration in all_configurations:
    row = dict(zip(configurations.keys(), configuration))
    row_list.append(row)

movement_df = pd.DataFrame(row_list)

movement_df["fname"] = movement_df.apply(config_to_name, axis=1)

movement_df

,laterality,localisation,movement,direction,fname
0,L,Head,X,For,L_Head_X_For
1,L,Head,X,Back,L_Head_X_Back
2,L,Head,Y,For,L_Head_Y_For
3,L,Head,Y,Back,L_Head_Y_Back
4,L,Head,Z,For,L_Head_Z_For
5,L,Head,Z,Back,L_Head_Z_Back
6,L,Arm,X,For,L_Arm_X_For
7,L,Arm,X,Back,L_Arm_X_Back
8,L,Arm,Y,For,L_Arm_Y_For
9,L,Arm,Y,Back,L_Arm_Y_Back


# CogShifting

## Params

In [24]:
#Global params
video_dir = "robot_stimuli"
subject = 0
sequence_size = 4
n_cues = [1,2,3]

#Main task
n_block = 3
n_trial = 9

#Training
training_block = 1
training_trial = 3

In [25]:
configurations = {
    'robot':{
    'laterality' : ['L', 'R'],
    'localisation' : ['Head', 'Arm', 'Torso', 'Base'],
    'movement' : ['X', 'Y', 'Z'],
    'direction' : ['For', 'Back']
    },

    'letter' : {
    # 'color':['#a1c9f4', '#ff9f9b'],
    'color':['blue', 'red'],
    'position':['left', 'top', 'right', 'bottom'],
    'letter': [elem for elem in string.ascii_uppercase if elem not in 'AEIOUY'],
    'orientation': ['0', '45'],
    },
}

## Time estimation

In [26]:
#Global times
block_intro_duration = 60
cue_duration = 2
inter_trial = 1

#Letter times
letter_stimuli_duration = 2

#Robot times
robot_stimuli_duration = 3

robot_trial_duration = inter_trial+cue_duration+(robot_stimuli_duration*sequence_size)
letter_trial_duration = inter_trial+cue_duration+(letter_stimuli_duration*sequence_size)

robot_block_duration = robot_trial_duration*n_trial
letter_block_duration = letter_trial_duration*n_trial

print(f"A total of {n_block*n_trial} trials for each condition ({len(n_cues)}) and each stimuli type (2)")
print(f"Each condition ({len(n_cues)}) with letter stimuli last {strftime('%X', gmtime(letter_block_duration))}")
print(f"Each condition ({len(n_cues)}) with robot stimuli last {strftime('%X', gmtime(robot_block_duration))}")

print(f"Estimated robot duration {strftime('%X', gmtime(robot_block_duration*n_block))} / {strftime('%X', gmtime((robot_block_duration+block_intro_duration)*n_block))}")
print(f"Estimated letter duration {strftime('%X', gmtime(letter_block_duration*n_block))} / {strftime('%X', gmtime((letter_block_duration+block_intro_duration)*n_block))}")

print(f"Estimated total duration {strftime('%X', gmtime((robot_block_duration+letter_block_duration)*n_block))} / {strftime('%X', gmtime((robot_block_duration+letter_block_duration+2*block_intro_duration)*n_block))}")

A total of 27 trials for each condition (3) and each stimuli type (2)
Each condition (3) with letter stimuli last 00:01:39
Each condition (3) with robot stimuli last 00:02:15
Estimated robot duration 00:06:45 / 00:09:45
Estimated letter duration 00:04:57 / 00:07:57
Estimated total duration 00:11:42 / 00:17:42


## Utils

In [27]:
def get_block_df(condition_df_list):
    condition_sequence =  [df['condition'].unique()[0] for df in condition_df_list]

    counter_dict = defaultdict(int)
    block_sequence = []
    for condition in condition_sequence:
        counter_dict[condition] += 1
        block_sequence.append(counter_dict[condition])
    
    for i, df in enumerate(condition_df_list):
        df['block'] = block_sequence[i]
        df['is_block_beginning'] = [True if i==0 else False for i in range(len(df))]

    return pd.concat(condition_df_list).reset_index(drop=True)

In [43]:
def get_cogshifting_df(selected_parameters, cue_condition, n_trial, configuration):

    sequences = []
    for i in range(n_trial):
        
        bricks = pd.DataFrame({key:random.sample(value, k=2) for key, value in configuration.items() if key in selected_parameters})
        normal, odd = bricks.loc[0], bricks.loc[1]

        trial = pd.Series(dtype='object')
        #Each stimuli in the sequence has normal parameters except at its position : 1st stimuli has an odd value for 1st parameter, 2nd stimuli for 2nd parameter, etc... + one full normal
        trial['sequence'] = ['_'.join([odd[key] if key==parameter else normal[key] if key in normal else random.choice(value) for key, value in configuration.items()]) for parameter in selected_parameters] + ['_'.join([normal[key] if key in normal else random.choice(value) for key, value in configuration.items()])]
        #Keep track of which stimuli is odd for which parameter
        trial['odd_ones'] = {elem:trial['sequence'][i] for i, elem in enumerate(selected_parameters)} 

        #Randomize stimuli order in the sequence
        trial['sequence'] = random.sample(trial['sequence'], k=len(trial['sequence']))
  
        sequences.append(trial)

    
    sequence_df = pd.DataFrame(sequences)

    sequence_df['parameter'] = selected_parameters*int(len(sequence_df)/len(selected_parameters))
    sequence_df['choices'] = sequence_df['parameter'].apply(lambda x : sorted([x] + random.sample([elem for elem in selected_parameters if elem!=x], k=cue_condition-1)))

    sequence_df['target'] = sequence_df.apply(lambda x : x['odd_ones'][x['parameter']], axis=1)
    sequence_df['target_idx'] = sequence_df.apply(lambda x : x['sequence'].index(x['target']), axis=1)
    sequence_df['condition'] = cue_condition
    
    sequence_df = sequence_df.sample(frac=1).reset_index(drop=True)

    return sequence_df


item_type = 'robot'
cue_condition = 2
stimuli_type = 'letter'

selected_parameters_dict = {
    'robot':['laterality', 'localisation', 'movement'],
    'letter':['color', 'position', 'orientation']
}

cogshifting_df = get_cogshifting_df(selected_parameters_dict[item_type], cue_condition, n_trial, configuration=configurations[item_type])

## Computation

In [90]:
def get_conditions(subject, n_trial, cue_conditions, n_block, stimuli_order, selected_parameters, configurations, video_dir, verbose=False):
    df_list = []

    for block in range(n_block):
        for stimuli_type in stimuli_order:
            for cue_condition in sample(cue_conditions, len(cue_conditions)):
                if verbose:
                    print(f"Cue {cue_condition}, block {block}, {stimuli_type} stimuli")
                df = get_cogshifting_df(selected_parameters[stimuli_type], cue_condition, n_trial, configuration=configurations[stimuli_type])

                #Metadata
                df['stimuli'] = stimuli_type
                df['block'] = block+1
                df['is_block_beginning'] = [True if i==0 else False for i in range(len(df))]
                df['is_block_ending'] = [True if i==(n_trial-1) else False for i in range(len(df))]

                if verbose:
                    if cue_condition==2:
                        count_serie = df['choices'].value_counts()
                        print([f"{index} : {count_serie.iloc[i]}" for i, index in enumerate(count_serie.index)])
                        # display(df)
                        print('-------------------------------------------------------------')

                df_list.append(df)

                
    
    cogshifting_df = get_block_df(df_list)
    cogshifting_df['sequence_fnames'] = cogshifting_df.apply(lambda x : [f"{video_dir}\\{elem}.mp4" if x['stimuli']=='robot' else f"{video_dir}\\\\filler.mp4" for elem in x['sequence']], axis=1)
    cogshifting_df['target_fname'] = cogshifting_df.apply(lambda x : f"{video_dir}\\{x['target']}.mp4" if x['stimuli']=='robot' else f"{video_dir}\\\\filler.mp4", axis=1)
    cogshifting_df['subject'] = subject

    return cogshifting_df

In [91]:
def get_training(subject, n_trial, cue_conditions, n_block, stimuli_order, selected_parameters, configurations, video_dir, verbose=False):
    df = get_conditions(subject=subject, n_trial=n_trial, cue_conditions=cue_conditions, n_block=n_block, 
                                    stimuli_order=stimuli_order, selected_parameters=selected_parameters, configurations=configurations, video_dir=video_dir, verbose=verbose)
    
    df['stimuli'] = pd.Categorical(df['stimuli'], categories=['letter', 'robot'], ordered=True)
    # df['condition'] = pd.Categorical(df['condition'], categories=[1?2], ordered=True)
    df = df.sort_values(by=['stimuli', 'condition'])
    
    return df

In [92]:
selected_parameters = {
    'robot':['laterality', 'localisation', 'movement'],
    'letter':['color', 'position', 'orientation']
}

stimuli_order = ['letter', 'robot']
cue_conditions = [1,2,3]

In [93]:
cogshifting_df = get_conditions(subject=subject, n_trial=n_trial, cue_conditions=cue_conditions, n_block=n_block, 
                                    stimuli_order=stimuli_order, selected_parameters=selected_parameters, configurations=configurations, video_dir=video_dir, verbose=True)

Cue 1, block 0, letter stimuli
Cue 3, block 0, letter stimuli
Cue 2, block 0, letter stimuli
["['color', 'position'] : 3", "['orientation', 'position'] : 3", "['color', 'orientation'] : 3"]
-------------------------------------------------------------
Cue 2, block 0, robot stimuli
["['laterality', 'localisation'] : 4", "['localisation', 'movement'] : 3", "['laterality', 'movement'] : 2"]
-------------------------------------------------------------
Cue 1, block 0, robot stimuli
Cue 3, block 0, robot stimuli
Cue 2, block 1, letter stimuli
["['color', 'orientation'] : 5", "['color', 'position'] : 3", "['orientation', 'position'] : 1"]
-------------------------------------------------------------
Cue 3, block 1, letter stimuli
Cue 1, block 1, letter stimuli
Cue 1, block 1, robot stimuli
Cue 3, block 1, robot stimuli
Cue 2, block 1, robot stimuli
["['laterality', 'localisation'] : 4", "['laterality', 'movement'] : 3", "['localisation', 'movement'] : 2"]
------------------------------------

In [94]:
cogshifting_training_df = get_training(subject=subject, n_trial=training_trial, cue_conditions=cue_conditions, n_block=training_block, 
stimuli_order=stimuli_order, selected_parameters=selected_parameters, configurations=configurations, video_dir=video_dir, verbose=True)

Cue 2, block 0, letter stimuli
["['color', 'position'] : 1", "['color', 'orientation'] : 1", "['orientation', 'position'] : 1"]
-------------------------------------------------------------
Cue 3, block 0, letter stimuli
Cue 1, block 0, letter stimuli
Cue 2, block 0, robot stimuli
["['laterality', 'movement'] : 2", "['localisation', 'movement'] : 1"]
-------------------------------------------------------------
Cue 1, block 0, robot stimuli
Cue 3, block 0, robot stimuli


## Saving

In [95]:
def safe_save(df, save_dir, fname, index=False):
    fname_versions = [elem for elem in os.listdir(save_dir) if elem.split('old_')[-1]==fname]
    sorted_fname_versions = sorted(fname_versions, key=len, reverse=True)

    for target_name in sorted_fname_versions:
        os.rename(f"{save_dir}\\{target_name}", f"{save_dir}\\old_{target_name}")

    
    df.to_csv(f"{save_dir}\\{fname}", index=index)

In [96]:
os.makedirs(subject_dir, exist_ok = True)

In [19]:
fname = f"conditions_cogshifting.csv"
safe_save(cogshifting_df, subject_dir, fname, index=False)

In [20]:
fname = f"conditions_cogshifting_short.csv"
safe_save(cogshifting_training_df.head(training_trial), subject_dir, fname, index=False)

In [21]:
fname = f"conditions_cogshifting_training.csv"
safe_save(cogshifting_training_df, subject_dir, fname, index=False)

## Multi-subject

In [97]:
subjects = range(24)

mode = 'hard'

selected_parameters = {
    'robot':['laterality', 'localisation', 'movement'],
    'letter':['color', 'position', 'orientation']
}

stimuli_order = ['letter', 'robot']
cue_conditions = [1,2,3]

for subject in subjects:
    subject_dir = f"{cdir}\\data\\subject_{subject}"

    if subject%2==0:
        stimuli_order = ['letter', 'robot']

    elif subject%2==1:
        stimuli_order = ['robot', 'letter']

    cogshifting_df = get_conditions(subject=subject, n_trial=n_trial, cue_conditions=cue_conditions, n_block=n_block, 
                                stimuli_order=stimuli_order, selected_parameters=selected_parameters, configurations=configurations, video_dir=video_dir, verbose=False)
    cogshifting_training_df = get_training(subject=subject, n_trial=training_trial, cue_conditions=cue_conditions, n_block=training_block, 
                                stimuli_order=stimuli_order, selected_parameters=selected_parameters, configurations=configurations, video_dir=video_dir, verbose=False)

    os.makedirs(subject_dir, exist_ok = True)

    if mode == 'hard':
        cogshifting_df.to_csv(f"{subject_dir}\\conditions_cogshifting.csv", index=False)
        cogshifting_training_df.head(training_trial).to_csv(f"{subject_dir}\\conditions_cogshifting_short.csv", index=False)
        cogshifting_training_df.to_csv(f"{subject_dir}\\conditions_cogshifting_training.csv", index=False)
    
    elif mode == 'safe':
        safe_save(cogshifting_df, subject_dir, "conditions_cogshifting.csv", index=False)
        safe_save(cogshifting_training_df.head(), subject_dir, "conditions_cogshifting_short.csv", index=False)
        safe_save(cogshifting_training_df, subject_dir, "conditions_cogshifting_training.csv", index=False)
